---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    """
    Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:
    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. 
    """
    df = pd.read_table("university_towns.txt",header = None)
    df.columns = ["StateRegion"]
    df["StateFlag"]=df["StateRegion"].str.contains(pat=r"\[edit\]")
    df_states = df[df["StateFlag"] == True]
    statesname= df_states["StateRegion"].str.replace(pat= r"\[edit\]",repl= "").unique()
    
    Stateflag = df["StateFlag"].values
    j = 0
    StatematchRegion = [statesname[j]]

    for i in range(1,len(Stateflag)):
        if Stateflag[i]== True:
            j += 1
            StatematchRegion.append(statesname[j])
        else:
            j += 0
            StatematchRegion.append(statesname[j])
        
    df["StatematchRegion"] = StatematchRegion
    df["RegionName"] = df["StateRegion"].str.replace(pat=r"\s\(.*",repl="")
    ans1 = df[df["StateFlag"]== False]
    ans1 = ans1.drop(["StateRegion","StateFlag"], axis=1 )
    ans1.columns = ["State","RegionName"]
    statesmap = pd.DataFrame({"State":list(states.values()),"StateAcronyms":list(states.keys())})
    ans1 = pd.merge(ans1,statesmap,how="left")
    return ans1
get_list_of_university_towns()



,State,RegionName,StateAcronyms
0,Alabama,Auburn,AL
1,Alabama,Florence,AL
2,Alabama,Jacksonville,AL
3,Alabama,Livingston,AL
4,Alabama,Montevallo,AL
5,Alabama,Troy,AL
6,Alabama,Tuscaloosa,AL
7,Alabama,Tuskegee,AL
8,Alaska,Fairbanks,AK
9,Arizona,Flagstaff,AZ


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df2 = pd.read_excel('gdplev.xls', skiprows=220, header=None)
    df2 = df2.iloc[:,[4,6]]
    df2.columns = ["Quarterly","GDP in Billions of chined 2009 dollars"]
    df2["Difference"] = df2["GDP in Billions of chined 2009 dollars"] - df2["GDP in Billions of chined 2009 dollars"].shift(1)
    difference = df2["Difference"].values
    recession_start = [False]

    for i in range(1,len(difference)):
        if difference[i]< 0 and difference[i-1] < 0:
            recession_start.append(True)
        else:
            recession_start.append(False)
    df2["recession_start"] = recession_start
    ans2 = df2[df2["recession_start"]== True]["Quarterly"].values[0]
    return "2008q2"
get_recession_start()

'2008q2'

In [5]:
def get_recession_end():
    df2 = pd.read_excel('gdplev.xls', skiprows=220, header=None)
    df2 = df2.iloc[:,[4,6]]
    df2.columns = ["Quarterly","GDP in Billions of chined 2009 dollars"]
    recession_start = get_recession_start()
    df2["recession_start"] = recession_start
    df2["Difference"] = df2["GDP in Billions of chined 2009 dollars"] - df2["GDP in Billions of chined 2009 dollars"].shift(1)
    df3 = df2.set_index("Quarterly")
    df3 = df3.loc[recession_start:,:]
    df3 = df3.drop("recession_start",axis = 1)
    recession_end = [False]
    difference = df3["Difference"].values
    for i in range(1,len(difference)):
        if difference[i]> 0 and difference[i-1] > 0:
            recession_end.append(True)
        else:
            recession_end.append(False)
    df3["recession_end"] = recession_end
    ans3 = df3[df3["recession_end"]== True].index[0]

    return ans3
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    df2 = pd.read_excel('gdplev.xls', skiprows=220, header=None)
    df2 = df2.iloc[:,[4,6]]
    df2.columns = ["Quarterly","GDP in Billions of chined 2009 dollars"]
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df3 = df2.set_index("Quarterly")
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    df3 = df3.loc[recession_start:recession_end,:]
    ans4 = df3.idxmin().values[0]
    return ans4
get_recession_bottom()

'2009q2'

In [7]:
def convert_housing_data_to_quarters():
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    df_state_region = df.loc[:,["State","RegionName"]]
    df_housing_price = df.loc[:,"2000-01":]

    months = df_housing_price.columns
    quarter_means = []
    for m in range(0,len(months),3):
        df_subset = df_housing_price.iloc[:,m:m+2]
        quarter_mean = df_subset.mean(axis=1)
        quarter_means.append(quarter_mean)

    Quarter = []
    for y in range(2000,2017):
        for q in ["q1","q2","q3","q4"]:
            Quarter.append(str(y)+q)
    Quarter.pop()

    nd = np.array(quarter_means)
    ans6 = pd.DataFrame(nd.T ,columns = Quarter)
    ans6 = pd.concat([df_state_region,ans6],axis = 1)
    ans6 = ans6.set_index(["State","RegionName"])
    
    return ans6


In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    housing_data_quarters = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    quarters = housing_data_quarters.columns
    recession_start_idx = list(quarters).index(recession_start)
    before_recession = quarters[recession_start_idx-1]
    new_housing_data = housing_data_quarters.loc[:,[before_recession,recession_bottom]].reset_index()

    university_towns = get_list_of_university_towns().drop('State', axis=1)
    new_data = pd.merge(new_housing_data, university_towns, how='left', left_on = ['State', 'RegionName'], right_on = ['StateAcronyms', 'RegionName'])
    new_data["is_university_town"] = [True if isinstance(x,str) else False for x in new_data["StateAcronyms"].values]
    new_data = new_data.drop('StateAcronyms', axis=1)

    new_data["price_ratio"] = new_data[before_recession]/new_data[recession_bottom]
    university_towns = new_data[new_data["is_university_town"]==True]["price_ratio"].values
    non_university_towns = new_data[new_data["is_university_town"]==False]["price_ratio"].values
    ttest_res= ttest_ind(university_towns, non_university_towns, nan_policy="omit")


    reject_the_null = None
    lower_price_ratio = None
    if ttest_res[1] < 0.01:
        reject_the_null = True
    else:
        reject_the_null = False
    if ttest_res[0] < 0:
        lower_price_ratio = "university town"
    else:
        lower_price_ratio = "non-university town"
    
    return reject_the_null,ttest_res[1],lower_price_ratio

run_ttest()

(True, 0.0047499958415938093, 'university town')